In [1]:
import pickle
import pandas as pd
from typing import Callable
import torch

import circuits.eval_sae_as_classifier as eval_sae
import circuits.analysis as analysis
import circuits.test_board_reconstruction as test_board_reconstruction
import circuits.get_eval_results as get_eval_results

/home/rangell_umass_edu/miniconda3/envs/sae/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# For multi-GPU evaluation
from collections import deque
from joblib import Parallel, delayed

from circuits.utils import to_device

N_GPUS = 1
RESOURCE_STACK = deque([f"cuda:{i}" for i in range(N_GPUS)])

In [3]:
def initialize_dataframe() -> pd.DataFrame:
    return pd.DataFrame(
        columns=[
            "autoencoder_group_path",
            "autoencoder_path",
            "reconstruction_file",
            "eval_results_n_inputs",
            "l0",
            "l1_loss",
            "l2_loss",
            "frac_alive",
            "frac_variance_explained",
            "cossim",
            "l2_ratio",
            "num_alive_features",
            "board_reconstruction_board_count",
            "best_idx",
            "zero_L0",
            "zero_f1_score",
            "best_L0",
            "best_f1_score",
            "zero_num_true_positive_squares",
            "best_num_true_positive_squares",
            "zero_num_false_positive_squares",
            "best_num_false_positive_squares",
            "zero_percent_active_classifiers",
            "best_percent_active_classifiers",
            "zero_classifiers_per_token",
            "best_classifiers_per_token",
            "zero_classified_per_token",
            "best_classified_per_token",
        ]
    )

def append_results(
    eval_results: dict,
    board_reconstruction_results: dict,
    custom_functions: list[Callable],
    df: pd.DataFrame,
    autoencoder_group_path: str,
    autoencoder_path: str,
    reconstruction_file: str,
) -> pd.DataFrame:
    
    print(eval_results)

    for custom_function in custom_functions:
        function_name = custom_function.__name__

        best_idx = board_reconstruction_results[custom_function.__name__]["f1_score"].argmax()

        new_row = {
            "autoencoder_group_path": autoencoder_group_path,
            "autoencoder_path": autoencoder_path,
            "reconstruction_file": reconstruction_file,
            "eval_results_n_inputs": eval_results["hyperparameters"]['n_inputs'],
            "l0": eval_results['eval_results']["l0"],
            "l1_loss": eval_results['eval_results']["l1_loss"],
            "l2_loss": eval_results['eval_results']["l2_loss"],
            "frac_alive": eval_results['eval_results']["frac_alive"],
            "frac_variance_explained": eval_results['eval_results']["frac_variance_explained"],
            "cossim": eval_results['eval_results']["cossim"],
            "l2_ratio": eval_results['eval_results']["l2_ratio"],
            "num_alive_features": board_reconstruction_results["alive_features"].shape[0],
            "board_reconstruction_board_count": board_reconstruction_results[function_name]["num_boards"],
            "best_idx": best_idx.item(),
            "zero_L0": board_reconstruction_results["active_per_token"][0].item(),
            "best_L0": board_reconstruction_results["active_per_token"][best_idx].item(),
            "zero_f1_score": board_reconstruction_results[function_name]["f1_score"][0].item(),
            "best_f1_score": board_reconstruction_results[function_name]["f1_score"][
                best_idx
            ].item(),
            "zero_num_true_positive_squares": board_reconstruction_results[function_name][
                "num_true_positive_squares"
            ][0].item(),
            "best_num_true_positive_squares": board_reconstruction_results[function_name][
                "num_true_positive_squares"
            ][best_idx].item(),
            "zero_num_false_positive_squares": board_reconstruction_results[function_name][
                "num_false_positive_squares"
            ][0].item(),
            "best_num_false_positive_squares": board_reconstruction_results[function_name][
                "num_false_positive_squares"
            ][best_idx].item(),
            "zero_percent_active_classifiers": (
                board_reconstruction_results[function_name]["classifiers_per_token"][0]
                / board_reconstruction_results["active_per_token"][0]
            ).item(),
            "best_percent_active_classifiers": (
                board_reconstruction_results[function_name]["classifiers_per_token"][best_idx]
                / board_reconstruction_results["active_per_token"][best_idx]
            ).item(),
            "zero_classifiers_per_token": board_reconstruction_results[function_name][
                "classifiers_per_token"
            ][0].item(),
            "best_classifiers_per_token": board_reconstruction_results[function_name][
                "classifiers_per_token"
            ][best_idx].item(),
            "zero_classified_per_token": board_reconstruction_results[function_name][
                "classified_per_token"
            ][0].item(),
            "best_classified_per_token": board_reconstruction_results[function_name][
                "classified_per_token"
            ][best_idx].item(),
        }

        new_row_df = pd.DataFrame([new_row])
        df = pd.concat([df, new_row_df], ignore_index=True)

    return df

Basically, just set `autoencoder_group_paths` and various hyperparameters and run it. If you already ran, for example, `eval_sae_as_classifier` and don't want to run it again, set `run_eval_sae` to False. Note that in this case, `eval_results_n_inputs` must match in order for it to load the file saved from the previous run.

By default, we `save_results`, which means each of the 4 functions saves a `.pkl` file. By default, we also aggregate and format some of the results into a csv `output_file`. If you already have results `.pkl` files and want a csv, you can set all `run_...` to False, and it will load the results and put them into a csv. 

In [4]:
import importlib
importlib.reload(eval_sae)
importlib.reload(analysis)
importlib.reload(test_board_reconstruction)
importlib.reload(get_eval_results)
import circuits.chess_utils as chess_utils
importlib.reload(chess_utils)

# NOTE: This script makes a major assumption here: That all autoencoders in a given group are trained on chess XOR Othello
# We do this so we don't have to reconstruct the dataset for each autoencoder in the group
# autoencoder_group_paths = ["../autoencoders/othello_layer5_ef4/"]
# autoencoder_group_paths = ["../autoencoders/chess_layer5/"]
autoencoder_group_paths = ["../autoencoders/group-2024-05-07/"]


eval_sae_n_inputs = 1000
batch_size = 100
#device = "cuda"
model_path = "../models/"
save_results = True

eval_results_n_inputs = 1000
board_reconstruction_n_inputs = 1000

analysis_high_threshold = 0.95
analysis_low_threshold = 0.1
analysis_significance_threshold = 10

run_eval_results = True # We don't check for this as eval_results are pretty quick to collect

# To skip any of the following steps, set the corresponding variable to False
# The results must have been saved previously
run_eval_sae = True
run_analysis = True
run_board_reconstruction = True

dataset_size = max(eval_sae_n_inputs, eval_results_n_inputs, board_reconstruction_n_inputs)

if dataset_size == eval_results_n_inputs:
    dataset_size *= 2

for autoencoder_group_path in autoencoder_group_paths:
    othello = eval_sae.check_if_autoencoder_is_othello(autoencoder_group_path)

    indexing_functions = eval_sae.get_recommended_indexing_functions(othello)
    indexing_function = indexing_functions[0]

    custom_functions = eval_sae.get_recommended_custom_functions(othello)

    model_name = eval_sae.get_model_name(othello)

    precompute = True

    print("Constructing evaluation dataset")
    device = RESOURCE_STACK.pop()
    data = eval_sae.construct_dataset(othello, custom_functions, dataset_size, device, models_path=model_path)
    RESOURCE_STACK.append(device)
    del device
    
    folders = eval_sae.get_nested_folders(autoencoder_group_path)

    def full_eval_pipeline(autoencoder_path):
        
        # Grab a GPU off the stack to use
        device = RESOURCE_STACK.pop()
        
        # For debugging
        # if "ef=4_lr=1e-03_l1=1e-01_layer=5" not in autoencoder_path:
        #     continue

        # If this is set, everything below should be reproducible
        # Then we can just save results from 1 run, make optimizations, and check that the results are the same
        # The determinism is only needed for getting activations from the activation buffer for finding alive features
        torch.manual_seed(0)
        eval_results = get_eval_results.get_evals(
            autoencoder_path,
            eval_results_n_inputs,
            device,
            model_path,
            model_name,
            to_device(data.copy(), device),
            othello=othello,
            save_results=save_results,
        )


        expected_aggregation_output_location = eval_sae.get_output_location(
                autoencoder_path, n_inputs=eval_sae_n_inputs, indexing_function=indexing_function
            )
        
        if run_eval_sae:
            print("Aggregating", autoencoder_path)
            aggregation_results = (
                eval_sae.aggregate_statistics(
                    custom_functions=custom_functions,
                    autoencoder_path=autoencoder_path,
                    n_inputs=eval_sae_n_inputs,
                    batch_size=batch_size,
                    device=device,
                    model_path=model_path,
                    model_name=model_name,
                    data=to_device(data.copy(), device),
                    indexing_function=indexing_function,
                    othello=othello,
                    save_results=save_results,
                    precomputed=precompute,
                )
            )
        else:
            with open(expected_aggregation_output_location, "rb") as f:
                aggregation_results = pickle.load(f)

        expected_feature_labels_output_location = expected_aggregation_output_location.replace(
            "results.pkl", "feature_labels.pkl"
        )
        if run_analysis:
            feature_labels = analysis.analyze_results_dict(
                aggregation_results,
                output_path=expected_feature_labels_output_location,
                device=device,
                high_threshold=analysis_high_threshold,
                low_threshold=analysis_low_threshold,
                significance_threshold=analysis_significance_threshold,
                verbose=False,
                print_results=False,
                save_results=save_results,
            )
        else:
            with open(expected_feature_labels_output_location, "rb") as f:
                feature_labels = pickle.load(f)

        expected_reconstruction_output_location = expected_aggregation_output_location.replace(
            "results.pkl", "reconstruction.pkl"
        )

        if run_board_reconstruction:
            print("Testing board reconstruction")
            board_reconstruction_results = test_board_reconstruction.test_board_reconstructions(
                    custom_functions=custom_functions,
                    autoencoder_path=autoencoder_path,
                    feature_labels=feature_labels,
                    output_file=expected_reconstruction_output_location,
                    n_inputs=board_reconstruction_n_inputs,
                    batch_size=batch_size,
                    device=device,
                    model_name=model_name,
                    data=to_device(data.copy(), device),
                    othello=othello,
                    print_results=False,
                    save_results=save_results,
            )
        else:
            with open(expected_reconstruction_output_location, "rb") as f:
                board_reconstruction_results = pickle.load(f)

        df = initialize_dataframe()
        df = append_results(
            eval_results,
            board_reconstruction_results,
            custom_functions,
            df,
            autoencoder_group_path,
            autoencoder_path,
            expected_reconstruction_output_location,
        )

        print("Finished", autoencoder_path)

        # Save the dataframe after each autoencoder so we don't lose data if the script crashes
        output_file = autoencoder_path + "/" + "results.csv"
        df.to_csv(output_file)
        
        # Put the GPU back on the stack after we're done
        RESOURCE_STACK.append(device)
        return df
    
    dfs = Parallel(n_jobs=N_GPUS, require="sharedmem")(delayed(full_eval_pipeline)(autoencoder_path) for autoencoder_path in folders)

Constructing evaluation dataset
board_to_piece_state
Element size: 1 bytes
Number of elements: 425984000
Memory usage: 406.25 MB
board_to_pin_state
Element size: 1 bytes
Number of elements: 512000
Memory usage: 0.48828125 MB


/home/rangell_umass_edu/miniconda3/envs/sae/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


AggregatingAggregating ../autoencoders/group-2024-05-07/PAnnealTrainer-chess-alpha0.04466836154460907L_p^p/
 ../autoencoders/group-2024-05-07/GatedSAETrainer-chess-alpha1.0/
Aggregating ../autoencoders/group-2024-05-07/StandardTrainer-chess-alpha0.07943282276391983/
Aggregating ../autoencoders/group-2024-05-07/GatedSAETrainer-chess-alpha0.7079457640647888/
Aggregating ../autoencoders/group-2024-05-07/StandardTrainer-chess-alpha0.09440608322620392/


/home/rangell_umass_edu/miniconda3/envs/sae/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Aggregating ../autoencoders/group-2024-05-07/PAnnealTrainer-chess-alpha0.05011872947216034L_p^p/
Aggregating ../autoencoders/group-2024-05-07/StandardTrainer-chess-alpha0.11220184713602066/
Aggregating ../autoencoders/group-2024-05-07/PAnnealTrainer-chess-alpha0.03981072083115578L_p^p/


...



Aggregating statistics: 100%|██████████| 10/10 [00:21<00:00,  2.11s/it]


{'hyperparameters': {'n_inputs': 1000, 'context_length': 256}, 'eval_results': {'l2_loss': 2.1893229484558105, 'l1_loss': 23.305150985717773, 'l0': 7.521000385284424, 'frac_alive': 0.000459045433672145, 'frac_variance_explained': 0.9575729966163635, 'cossim': 0.950599193572998, 'l2_ratio': 0.9501383304595947}}
Finished ../autoencoders/group-2024-05-07/GatedAnnealTrainer-chess-alpha0.5623413324356079L_p^p/


/tmp/ipykernel_2513515/2329706948.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row_df], ignore_index=True)


Aggregating statistics:  80%|████████  | 8/10 [00:21<00:05,  2.91s/it]

Aggregating statistics:  90%|█████████ | 9/10 [00:24<00:02,  2.83s/it]

Aggregating statistics: 100%|██████████| 10/10 [00:27<00:00,  2.71s/it]


{'hyperparameters': {'n_inputs': 1000, 'context_length': 256}, 'eval_results': {'l2_loss': 1.9210543632507324, 'l1_loss': 26.373027801513672, 'l0': 11.141000747680664, 'frac_alive': 0.0006799927214160562, 'frac_variance_explained': 0.9678363800048828, 'cossim': 0.9638010859489441, 'l2_ratio': 0.9648488759994507}}
Finished ../autoencoders/group-2024-05-07/GatedAnnealTrainer-chess-alpha0.3981071710586548L_p^p/


/tmp/ipykernel_2513515/2329706948.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row_df], ignore_index=True)


In [7]:
# TODO: merge results.csv from all of the folders
df = pd.concat(dfs, axis=0, ignore_index=True)
df

/tmp/ipykernel_2513515/614852007.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, axis=0, ignore_index=True)


,autoencoder_group_path,autoencoder_path,reconstruction_file,eval_results_n_inputs,l0,l1_loss,l2_loss,frac_alive,frac_variance_explained,cossim,...,zero_num_true_positive_squares,best_num_true_positive_squares,zero_num_false_positive_squares,best_num_false_positive_squares,zero_percent_active_classifiers,best_percent_active_classifiers,zero_classifiers_per_token,best_classifiers_per_token,zero_classified_per_token,best_classified_per_token
0,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/StandardTrain...,../autoencoders/group-2024-05-07/StandardTrain...,1000,49.828003,16.864666,1.627392,0.003041,0.977771,0.974109,...,653280,0,34524,0,0.0,0.0,0.0,0.0,0.0,0.0
1,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/StandardTrain...,../autoencoders/group-2024-05-07/StandardTrain...,1000,49.828003,16.864666,1.627392,0.003041,0.977771,0.974109,...,14,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/StandardTrain...,../autoencoders/group-2024-05-07/StandardTrain...,1000,37.215000,15.704080,1.813791,0.002271,0.972351,0.967721,...,609570,0,30270,0,0.0,0.0,0.0,0.0,0.0,0.0
3,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/StandardTrain...,../autoencoders/group-2024-05-07/StandardTrain...,1000,37.215000,15.704080,1.813791,0.002271,0.972351,0.967721,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/StandardTrain...,../autoencoders/group-2024-05-07/StandardTrain...,1000,27.304001,14.537116,2.022861,0.001667,0.965702,0.959808,...,587846,0,27072,0,0.0,0.0,0.0,0.0,0.0,0.0
5,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/StandardTrain...,../autoencoders/group-2024-05-07/StandardTrain...,1000,27.304001,14.537116,2.022861,0.001667,0.965702,0.959808,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
6,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/PAnnealTraine...,../autoencoders/group-2024-05-07/PAnnealTraine...,1000,18.823000,20.337763,1.619309,0.001149,0.978854,0.974982,...,816315,0,32057,0,0.0,0.0,0.0,0.0,0.0,0.0
7,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/PAnnealTraine...,../autoencoders/group-2024-05-07/PAnnealTraine...,1000,18.823000,20.337763,1.619309,0.001149,0.978854,0.974982,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/PAnnealTraine...,../autoencoders/group-2024-05-07/PAnnealTraine...,1000,16.142000,19.322727,1.699897,0.000985,0.976602,0.972113,...,812246,867426,29864,24811,0.0,0.0,0.0,0.0,0.0,0.0
9,../autoencoders/group-2024-05-07/,../autoencoders/group-2024-05-07/PAnnealTraine...,../autoencoders/group-2024-05-07/PAnnealTraine...,1000,16.142000,19.322727,1.699897,0.000985,0.976602,0.972113,...,51,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Example of gathering top k contexts

In [ ]:
import torch
import circuits.chess_interp as chess_interp
importlib.reload(chess_interp)

torch.set_grad_enabled(False)

autoencoder_group_path = autoencoder_group_paths[0]

othello = eval_sae.check_if_autoencoder_is_othello(autoencoder_group_path)

indexing_functions = eval_sae.get_recommended_indexing_functions(othello)
indexing_function = indexing_functions[0]

custom_functions = eval_sae.get_recommended_custom_functions(othello)

model_name = eval_sae.get_model_name(othello)

device = RESOURCE_STACK.pop()
print("Constructing evaluation dataset")
data = eval_sae.construct_dataset(othello, custom_functions, dataset_size, device, models_path=model_path)


dataset_size = dataset_size * 2  # x2 to make sure we have enough data for loss_recovered()

data, ae_bundle, pgn_strings, encoded_inputs = eval_sae.prep_firing_rate_data(
    autoencoder_path, dataset_size, model_path, model_name, data, device, dataset_size, othello
)

dims = torch.tensor([10], device=device)
chess_interp.examine_dimension_chess(ae_bundle, 100, dims)

RESOURCE_STACK.append(device)
del device

In [ ]:
RESOURCE_STACK